<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/PML_sem_barrier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install spin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  spin
0 upgraded, 1 newly installed, 0 to remove and 22 not upgraded.
Need to get 559 kB of archives.
After this operation, 1,776 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 spin amd64 6.4.6+dfsg-2 [559 kB]
Fetched 559 kB in 2s (276 kB/s)
Selecting previously unselected package spin.
(Reading database ... 144618 files and directories currently installed.)
Preparing to unpack .../spin_6.4.6+dfsg-2_amd64.deb ...
Unpacking spin (6.4.6+dfsg-2) ...
Setting up spin (6.4.6+dfsg-2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!spin -V

Spin Version 6.4.6 -- 2 December 2016


In [ ]:
%%writefile sem.pml
#define wait(s) atomic{s>0->s--}
#define signal(s) s++

byte S=1

active[2] proctype P(){
    byte i;
    for (i:1..100){
      printf("%d NCS 1\n",_pid);
      printf("%d NCS 2\n",_pid);
      printf("%d NCS 3\n",_pid);
       wait(S)
      printf("%d CRITICAL 1\n",_pid);
      printf("%d CRITICAL 2\n",_pid);
      printf("%d CRITICAL 3\n",_pid);
      signal(S)
    }
}

Overwriting sem.pml


In [ ]:
!spin sem.pml

          1 NCS 1
          1 NCS 2
      0 NCS 1
          1 NCS 3
          1 CRITICAL 1
      0 NCS 2
      0 NCS 3
          1 CRITICAL 2
          1 CRITICAL 3
          1 NCS 1
          1 NCS 2
      0 CRITICAL 1
          1 NCS 3
      0 CRITICAL 2
      0 CRITICAL 3
          1 CRITICAL 1
          1 CRITICAL 2
          1 CRITICAL 3
      0 NCS 1
      0 NCS 2
          1 NCS 1
      0 NCS 3
          1 NCS 2
          1 NCS 3
      0 CRITICAL 1
      0 CRITICAL 2
      0 CRITICAL 3
          1 CRITICAL 1
      0 NCS 1
      0 NCS 2
          1 CRITICAL 2
      0 NCS 3
          1 CRITICAL 3
      0 CRITICAL 1
      0 CRITICAL 2
          1 NCS 1
          1 NCS 2
      0 CRITICAL 3
          1 NCS 3
          1 CRITICAL 1
          1 CRITICAL 2
      0 NCS 1
          1 CRITICAL 3
      0 NCS 2
      0 NCS 3
      0 CRITICAL 1
      0 CRITICAL 2
      0 CRITICAL 3
          1 NCS 1
      0 NCS 1
          1 NCS 2
      0 NCS 2
      0 NCS 3
          1 NCS 3
          1 CRIT

In [ ]:
%%writefile sem2.pml
#define wait(s) atomic{s>0->s--}
#define signal(s) s++

byte S=1

active[2] proctype P(){
    do
    ::
       wait(S)
       assert(S>=0)
      signal(S)
    od
}

Writing sem2.pml


In [ ]:
!spin -run sem2.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 20 byte, depth reached 2, errors: 0
        5 states, stored
        2 states, matched
        7 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.000	equivalent memory usage for states (stored*(State-vector + overhead))
    0.292	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage


unreached in proctype P
	sem2.pml:13, state 9, "-end-"
	(1 of 9 states)

pan: elapsed time 0 seconds


In [ ]:
%%writefile mergesort.pml
#define wait(s) atomic{s>0->s=false}
#define signal(s) s=true


int A[6]={3,5,2,4,1,6}
bool S1=false
bool S2=false

active proctype Sort1(){
    A[0]=2
    A[1]=3
    A[2]=5
    signal(S1)
}

active proctype Sort2(){
    A[3]=1
    A[4]=4
    A[5]=6
    signal(S2)
}

active proctype Merge(){
    wait(S1)
    wait(S2)
    printf("%d %d %d %d %d %d\n",A[0],A[1],A[2],A[3],A[4],A[5])
}


Overwriting mergesort.pml


In [ ]:
!spin mergesort.pml

              2 3 5 1 4 6
3 processes created


In [ ]:
%%writefile prodcons.pml
#define wait(s) atomic{s>0->s--}
#define signal(s)s++

#define push(p,t,e)atomic{p[t]=e;t++}
#define pop(p,t,e)atomic{t--;e=p[t]}

#define N 5

byte notFull=5
byte notEmpty=0

int cache[N]
byte top=0

active proctype Producer(){
    int d
    int i=0
    for(i,1..10)
      i++
      d=1000* _pid+i
      wait(notFull)
      append(buffer,d)
      signal(notEmpty)
    od
}

active proctype Consumer(){
    int d
    do
    ::
      wait(notEmpty)
      take(buffer,d)
      signal(notFull)
      printf("%d consuming %d\n",_pid,d)
}

Overwriting prodcons.pml


In [ ]:
%%script bash
spin prodcons.pml &
BGID=$!
sleep 1
kill $BGID

spin: prodcons.pml:23, Error: undeclared variable: append	saw ''(' = 40'


bash: line 4: kill: (1292) - No such process


In [ ]:
%%writefile barrier.pml
#define wait(s) atomic{s>0->s--}
#define signal(s) s++

#define N 5

byte count=0
byte mutex=1
byte barrier=0

active[N] proctype P(){
    printf("Randezvous\n")
    wait(mutex)
    cont++
    signal(mutex)

    if
    :: cont==N->signal(barrier);
    ::else ->skip
    fi

    wait(barrier)
    signal(barrier)

    printf("Continuando andoooooo")
}

Overwriting barrier.pml


In [ ]:
!spin barrier.pml

spin: barrier.pml:13, Error: undeclared variable: cont	saw 'operator: ++'
